# CUDA Kernels

CUDA kernels are arbitrary functions (not just element-wise) that run on the GPU in parallel by different CUDA threads.

- __Execution configuration__ of a kernel controls the grid where the kernel is launched
- The __grid__ consists of multiple blocks of threads
- CUDA kernels use `out` array and don't require explicit type signature.

__References__
- [Introduction](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#introduction)
- [Programming-model](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model)
- [CUDA Environment Variables](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars)
- [Atomic operations](https://numba.pydata.org/numba-doc/dev/cuda/intrinsics.html#supported-atomic-operations)

__Hardware__
- [GEFORCE GTX 1050 specs](https://www.nvidia.com/en-in/geforce/products/10series/geforce-gtx-1050/)
- [Compute capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities)
- [Hardware Implementation](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#hardware-implementation)


- In Pascal, an SM (streaming multiprocessor) consists of 128 CUDA cores.
- Blocks are mapped to multiprocessors. The multiprocessor creates, manages, schedules, and executes threads in groups of 32 parallel threads called __warps__.

__GEFORCE GTX 1050 specs__

Parameter | Value
--- | ---
NVIDIA CUDA® Cores | 640
Memory Speed | 7 Gbps
Memory Interface Width | 128-bit
Memory Bandwidth (GB/sec) | 112

In [ ]:
from numba import cuda, types
import numpy as np

In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    idx = cuda.grid(1) # unique thread index within the entire grid
    # grid(1) one dimensional thread grid, returns a single value
    # same as: idx = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    out[idx] = x[idx] + y[idx]

n = 1280
x = np.ones(n).astype(np.int32)
y = np.ones_like(x)

x_dev = cuda.to_device(x)
y_dev = cuda.to_device(y)
d_out = cuda.device_array_like(x_dev)

# execution configuration
# if blocks_per_grid * threads_per_block < n, some entries of the output will be empty
blocks_per_grid = 10 # multiplicative of number of SMs
# more blocks -> high CUDA kernel launch overhead
threads_per_block = 128 # multiplicative of 32

add_kernel[blocks_per_grid, threads_per_block](x_dev, y_dev, d_out)
print(d_out.copy_to_host()) # implicit cuda.synchronize()

## Grid Stride Loops
When dealing with large datasets and possibly trying to use large grid sizes (that'll result in a high launch overhead), it's possible to use grid stride loops as below. For more details, check [NVIDIA Developer Blog](https://devblogs.nvidia.com/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/)

more elements than threads

In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    idx_in_grid = cuda.grid(1) # (1) one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # same as: cuda.blockDim.x * cuda.gridDim.x
    for idx in range(idx_in_grid, x.shape[0], stride):
        out[idx] = x[idx] + y[idx]
        print(idx) # shown only when run in the terminal with `@cuda.jit(debug=True)`

n = 12800
x = np.ones(n).astype(np.int32)
y = np.ones_like(x)

x_dev = cuda.to_device(x)
y_dev = cuda.to_device(y)
d_out = cuda.device_array_like(x_dev)

blocks_per_grid = 10
threads_per_block = 128

add_kernel[blocks_per_grid, threads_per_block](x_dev, y_dev, d_out)
print(d_out.copy_to_host())

## Working with 2-dimensional datasets

In [ ]:
@cuda.jit
def add_kernel(x, y, out):
    idy,idx = cuda.grid(2)
    # grid(2) two dimensional thread grid, returns two values
    # stride_y, stride_x = cuda.gridsize(2)

    # out[idy,idx] = x[idy,idx] + y[idy,idx]
    out[idy,idx] = idy

n = 4
x = 2 * np.ones((n,n)).astype(np.int32)
y = np.ones_like(x)

x_dev = cuda.to_device(x)
y_dev = cuda.to_device(y)
d_out = cuda.device_array_like(x_dev)

blocks_per_grid = (2,2)
threads_per_block = (2,2)

add_kernel[blocks_per_grid, threads_per_block](x_dev, y_dev, d_out)
print(d_out.copy_to_host())

## Shared memory
Shared memory is a programmer defined cache of limited size & it's shared between all threads in a block.

- Shared memory is stored in what is called __banks__
- [Memory Hierarchy](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#memory-hierarchy)

In [ ]:
@cuda.jit
def fun_shared(x, y):
    idx = cuda.grid(1)
    shared_mem = cuda.shared.array(2, dtype=types.int32) # 2 elements in shared memory

    shared_mem[idx] = x[idx] # from global to shared memory
    cuda.syncthreads() # sync all threads in a block
    y[idx] = shared_mem[idx] * 2 # shared to global

x = np.arange(3,5).astype(np.int32)
y = np.zeros_like(x)

d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

%timeit fun_shared[1, 4](d_x, d_y); cuda.synchronize()
print(d_y.copy_to_host())


## Debugging
- Single threaded scenario
```
%time cuda_fun[1, 1](a,b,out); cuda.synchronize()
%timeit cuda_fun[1, 1](a,b,out); cuda.synchronize()
```
- Use print inside a kernel with `@cuda.jit(debug=True)`
- [Numba's CUDA Simulator](https://numba.pydata.org/numba-doc/dev/cuda/simulator.html). Inside a kernel include something similar to
```
if idy == 1 and idx == 3:
        from pdb import set_trace; set_trace(
```
then run
```
NUMBA_ENABLE_CUDASIM=1 python cuda_fun.py
```
[Python Debugger](https://docs.python.org/3/library/pdb.html) commands
```
n(ext)
p expression: Evaluate the expression in the current context and print its value.
q(uit)
```
- Memory issues such as accessing invalid memory addresses may be checked using [Cuda Memcheck](http://docs.nvidia.com/cuda/cuda-memcheck/)
```
cuda-memcheck python cuda_fun.py
```